# Fast Fourier Transform

Fourier Transform is an important method to convert data from time domain (time series) into the frequency domain, where the contributions from each frequency is represented by its amplitude. Fast Fourier Transform (FFT) is an algorithm to compute the discrete Fourier Transform efficiently.

Consider the following signal:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack

np.random.seed(1234)

time_step = 0.02
period = 5.

t = np.arange(0, 20, time_step)
X = (np.sin(2 * np.pi / period * t) + 0.5 * np.random.randn(t.size))

plt.figure(figsize=(12, 5))
plt.plot(t, X, 'g-', label='Original signal')
plt.plot(t, np.sin(2 * np.pi / period * t), 'r-', label='"main" signal')
plt.xlim([0, 20])
plt.legend()

This signal is composed of a "main" signal (shown in red) of period 5 plus some noise. Suppose we do not know about this main signal, but want to filter out the high-frequency noise from the signal.

In [ ]:
# The FFT of the signal
sig_fft = fftpack.fft(X)

# And the power (sig_fft is of complex dtype)
power = np.abs(sig_fft)

# The corresponding frequencies
sample_freq = fftpack.fftfreq(sig.size, d=time_step)

# Plot the FFT power
plt.figure(figsize=(8, 5))
plt.plot(sample_freq, power)
plt.xlabel('Frequency [Hz]')
plt.ylabel('plower')

# Find the peak frequency: we can focus on only the positive frequencies
pos_mask = np.where(sample_freq > 0)
freqs = sample_freq[pos_mask]
peak_freq = freqs[power[pos_mask].argmax()]

# Check that it does indeed correspond to the frequency that we generate
# the signal with
np.allclose(peak_freq, 1./period)

# An inner plot to show the peak frequency
axes = plt.axes([0.55, 0.3, 0.3, 0.5])
plt.title('Peak frequency')
plt.plot(freqs[:8], power[:8], 'ko-')
plt.setp(axes, yticks=[])

# scipy.signal.find_peaks_cwt can also be used for more advanced
# peak detection

Now we can remove the high-frequency noise, by setting the magnitudes of other non-essential frequencies to zero in the frequency (FFT'd) domain.

In [ ]:
high_freq_fft = sig_fft.copy()

print('peak frequency = ', peak_freq)

# kill the high frequency components in the frequency domain (set them to zero)
high_freq_fft[np.abs(sample_freq) > peak_freq] = 0

# transform from frequency domain to time domain
filtered_sig = fftpack.ifft(high_freq_fft).real

plt.figure(figsize=(12, 5))
plt.plot(t, sig, label='Original signal')
plt.plot(t, filtered_sig, 'r-', linewidth=3, label='Filtered signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.legend(loc='best')

**Exercise**

What if we chop off frequencies higher than `10*peak_freq` (instead of `1*peak_freq`)?

In [ ]:
high_freq_fft = sig_fft.copy()

# kill the high frequency components in the frequency domain (set them to zero)
high_freq_fft[np.abs(sample_freq) > 10*peak_freq] = 0

# transform from frequency domain to time domain
filtered_sig = fftpack.ifft(high_freq_fft).real

plt.figure(figsize=(12, 5))
plt.plot(t, sig, label='Original signal')
plt.plot(t, filtered_sig, 'r-', linewidth=3, label='Filtered signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.legend(loc='best')

Observe the the filtered signals contain *some* high frequencies components.

## Filtering image using FFT

Consider the following image:

In [ ]:
im = plt.imread('moonlanding.png')
im.shape, im.dtype

In [ ]:
plt.imshow(im, plt.cm.gray)

Now we can take 2-dimensional FFT to transform the image into the frequency domain

In [ ]:
im_fft = fftpack.fft2(im)

# Show the results
def plot_spectrum(im_fft):
    from matplotlib.colors import LogNorm
    # A logarithmic colormap
    plt.imshow(np.abs(im_fft), norm=LogNorm(vmin=5))
    plt.colorbar()

plt.figure()
plot_spectrum(im_fft)
plt.title('Fourier transform')

Now we can take out the high-frequency components, similar to the 1D example discussed earlier:

In [ ]:
keep_fraction = 0.1

# Call ff a copy of the original transform. Numpy arrays have a copy
# method for this purpose.
im_fft2 = im_fft.copy()

# Set r and c to be the number of rows and columns of the array.
r, c = im_fft2.shape

# Set to zero all rows with indices between r*keep_fraction and
# r*(1-keep_fraction):
im_fft2[int(r*keep_fraction):int(r*(1-keep_fraction))] = 0

# Similarly with the columns:
im_fft2[:, int(c*keep_fraction):int(c*(1-keep_fraction))] = 0

plt.figure()
plot_spectrum(im_fft2)
plt.title('Filtered Spectrum')

In [ ]:
im_new = fftpack.ifft2(im_fft2).real

plt.figure()
plt.imshow(im_new, plt.cm.gray)
plt.title('Reconstructed Image')

**Exercise** 

Can you come up with a new filtering criterion that can produce a better image than the reconstructed image above? Experiment!